Blurb from proposal : In a multi-agent system individual agents with different personas can communicate, collaborate, add additional context, and do work in parallel. They often have special rules for how to work as a team, decide things together, and handle disagreements.

Disclaimer (We can Remove or modify this once we rename the agents to build on our complex usecase): This notebooks uses and explains code snippets from Autogen Official Docs to demonstrate various Multi-Agent Usecases along with important concepts. Please refer to https://microsoft.github.io/autogen/docs/ for information.

In [ ]:
#Install packages
# !pip install pyautogen
# !pip install openai
# !pip install chroma==0.4.24

In [1]:
import os
import autogen
from dotenv import load_dotenv

load_dotenv()
# Save your OPENAI_API_KEY in a .env file in the same directory as this script
llm_config_dict = {"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]}

In [ ]:
# Construct a Single Agent and Run it

In [ ]:
# Construct a Single Agent with Persona and Run it

In [ ]:
# Construct a Single Agent with a tool and a Persona and Run it

# Two-agent chat: the simplest form of conversation pattern where two agents chat with each other using Autogen.

In [19]:
import os

from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config=llm_config_dict,
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a math teacher.",
    llm_config=llm_config_dict,
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="How to prove triangle inequality",
    summary_method="reflection_with_llm",
    max_turns=2,
)

[autogen.oai.client: 09-20 15:16:07] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 15:16:07] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Student_Agent (to Teacher_Agent):

How to prove triangle inequality

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

To prove the triangle inequality in geometry, you need to show that the sum of the lengths of any two sides of a triangle is always greater than the length of the third side. Let's say we have a triangle with sides a, b, and c.

The triangle inequality states that:

a + b > c
a + c > b
b + c > a

To prove this, you can use the properties of triangles such as the fact that the sum of the lengths of any two sides of a triangle is always greater than the length of the third side. 

1. 

Let's look at the summary. It is contained within the chat_result object of type ChatResult, which was returned by the initiate_chat method.

In [20]:
print(chat_result.summary)

The triangle inequality states that the sum of the lengths of any two sides of a triangle is always greater than the length of the third side. To prove this, you can use the properties of triangles and show that in a triangle, the sum of any two sides must be greater than the third side.


In the given example, the summary method is configured to reflection_with_llm, which processes a list of conversation messages and summarizes them by invoking an LLM. Initially, the summary method attempts to utilize the recipient’s LLM; if it is unavailable, it defaults to the sender’s LLM. Here, the recipient is “Teacher_Agent” and the sender is “Student_Agent”. The input prompt for the LLM is the following default prompt:

In [14]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


# Sequential chat: 
a series of conversations between two agents, linked by a carryover mechanism that transfers the summary of the prior chat to the context of the subsequent chat.

In [17]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config_dict,
    human_input_mode="NEVER",
)

[autogen.oai.client: 09-18 14:01:14] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-18 14:01:14] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-18 14:01:15] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-18 14:01:15] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-18 14:01:16] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [18]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Context: 
16

--------------------------------------------------------------------------------
Mul

In [19]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  You're welcome! If you need any more assistance with numbers, feel free to ask.
Fourth Chat Summary:  You're welcome! If you have any more numbers to divide by 2 or any other math-related questions, feel free to ask.


# Tool Use
In this section, we'll demonstrate tool-use capability of the agentic system. Web Search and RAG are popular capabilities used by people every day, we show how to use AutoGen to do:

* Web search using Bing API. 
* Web scraping using Apify API
* Question and answering with RAG.

More info can be found here : https://microsoft.github.io/autogen/docs/tutorial/tool-use 

## Web Search using Bing API

In [13]:
# Put your Bing API key in the .env file in the same directory as this script
BING_API_KEY = os.environ["BING_API_KEY"]

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

In [14]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": BING_API_KEY},
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

[autogen.oai.client: 09-20 19:18:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:18:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:18:43] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:18:44] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:18:44] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:18:45] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:18:45] {184} WARNING - The API key specified is not a valid OpenAI format; i

In [15]:
task1 = """
What is the history of Boise, Idaho?
"""

user_proxy.initiate_chat(web_surfer, message=task1)

user_proxy (to web_surfer):


What is the history of Boise, Idaho?


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
web_surfer (to user_proxy):

Address: bing: history of Boise, Idaho
Title: history of Boise, Idaho - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'history of Boise, Idaho' found 10 results:

## Web Results
1. [Boise, Idaho - Wikipedia](https://en.wikipedia.org/wiki/Boise,_Idaho)
Boise (locally / ˈ b ɔɪ s i / ⓘ BOY-see, [5] is the capital and most populous city in the U.S. state of Idaho and is the county seat of Ada County.As of the 2020 census, there were 235,684 people residing in the city.On the Boise River in southwestern Idaho, it is 41 miles (66 km) east of the Oregon border and 110 miles (177 km) north of the Nevada border. . The downtown area's elevation ...

2. [Boise, Idaho: The Origin Story of Boise and the Name](https://t

ChatResult(chat_id=None, chat_history=[{'content': '\nWhat is the history of Boise, Idaho?\n', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Address: bing: history of Boise, Idaho\nTitle: history of Boise, Idaho - Search\nViewport position: Showing page 1 of 1.\n=======================\nA Bing search for 'history of Boise, Idaho' found 10 results:\n\n## Web Results\n1. [Boise, Idaho - Wikipedia](https://en.wikipedia.org/wiki/Boise,_Idaho)\nBoise (locally / ˈ b ɔɪ s i / ⓘ BOY-see, [5] is the capital and most populous city in the U.S. state of Idaho and is the county seat of Ada County.As of the 2020 census, there were 235,684 people residing in the city.On the Boise River in southwestern Idaho, it is 41 miles (66 km) east of the Oregon border and 110 miles (177 km) north of the Nevada border. . The downtown area's elevation ...\n\n2. [Boise, Idaho: The Origin Story of Boise and the Name](https://thisisboise.com/boise-idaho-origin-story-and-the-name/)\nFollow This Is Boise. Bo

In [16]:
task2 = "Click on the first link."
user_proxy.initiate_chat(web_surfer, message=task2)

user_proxy (to web_surfer):

Click on the first link.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION navigational_web_search...
web_surfer (to user_proxy):

Address: https://en.wikipedia.org/wiki/Boise,_Idaho
Title: Boise, Idaho - Wikipedia
Viewport position: Showing page 1 of 42.
# Boise, Idaho

Capital city of Idaho, United States
"Boise" redirects here. For other uses, see [Boise (disambiguation)](/wiki/Boise_(disambiguation) "Boise (disambiguation)").
Not to be confused with [Boise County, Idaho](/wiki/Boise_County,_Idaho "Boise County, Idaho").

State capital city in Idaho, United States

| Boise, Idaho | |
| --- | --- |
| [State capital city](/wiki/List_of_capitals_in_the_United_States "List of capitals in the United States") | |
| [Idaho State Capitol](/wiki/Idaho_State_Capitol "Idaho State Capitol")[US Bank Building](/wiki/US_Bank_Plaza_(Boise) "US Bank Plaza (Boise)")[Eighth \& Main](

ChatResult(chat_id=None, chat_history=[{'content': 'Click on the first link.', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Address: https://en.wikipedia.org/wiki/Boise,_Idaho\nTitle: Boise, Idaho - Wikipedia\nViewport position: Showing page 1 of 42.\n=======================\n# Boise, Idaho\n\nCapital city of Idaho, United States\n"Boise" redirects here. For other uses, see [Boise (disambiguation)](/wiki/Boise_(disambiguation) "Boise (disambiguation)").\nNot to be confused with [Boise County, Idaho](/wiki/Boise_County,_Idaho "Boise County, Idaho").\n\nState capital city in Idaho, United States\n\n| Boise, Idaho | |\n| --- | --- |\n| [State capital city](/wiki/List_of_capitals_in_the_United_States "List of capitals in the United States") | |\n| [Idaho State Capitol](/wiki/Idaho_State_Capitol "Idaho State Capitol")[US Bank Building](/wiki/US_Bank_Plaza_(Boise) "US Bank Plaza (Boise)")[Eighth \\& Main](/wiki/Eighth_%26_Main "Eighth & Main")[Clearwater Analytics](/wiki/Clearwat

In [17]:
task3 = "What is the population of Boise, Idaho?"
user_proxy.initiate_chat(web_surfer, message=task3)

user_proxy (to web_surfer):

What is the population of Boise, Idaho?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION answer_from_page...
web_surfer (to user_proxy):

Boise, Idaho, the capital city of Idaho, has a population of 235,684 as of the 2020 census. The city is located in southwestern Idaho, about 41 miles east of the Oregon border and 110 miles north of the Nevada border. Boise experiences a semi-arid continental climate with four distinct seasons. Summers are hot and dry with temperatures reaching 100°F on average nine days a year. Winters are moderately cold with occasional lows below 0°F. Spring and fall are mild. Boise receives an average of 12 inches of precipitation annually, with snowfall averaging 18 inches.

The city is divided into various neighborhoods, including Downtown Boise, known for its cultural center and small businesses; the North End, with older homes and tree-line

ChatResult(chat_id=None, chat_history=[{'content': 'What is the population of Boise, Idaho?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'Boise, Idaho, the capital city of Idaho, has a population of 235,684 as of the 2020 census. The city is located in southwestern Idaho, about 41 miles east of the Oregon border and 110 miles north of the Nevada border. Boise experiences a semi-arid continental climate with four distinct seasons. Summers are hot and dry with temperatures reaching 100°F on average nine days a year. Winters are moderately cold with occasional lows below 0°F. Spring and fall are mild. Boise receives an average of 12 inches of precipitation annually, with snowfall averaging 18 inches.\n\nThe city is divided into various neighborhoods, including Downtown Boise, known for its cultural center and small businesses; the North End, with older homes and tree-lined streets; and Southeast Boise, home to Boise State University and Micron Technology. Other neighborhoods 

## Web Scraping Using Apify Tool

In [18]:
# Put your Apify API key in the .env file in the same directory as this script
APIFY_API_KEY = os.environ["APIFY_API_KEY"]

In [19]:
from apify_client import ApifyClient
from typing_extensions import Annotated


def scrape_page(url: Annotated[str, "The URL of the web page to scrape"]) -> Annotated[str, "Scraped content"]:
    # Initialize the ApifyClient with your API token
    client = ApifyClient(token=APIFY_API_KEY)

    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    run = client.actor("aYG0l9s7dbB7j3gbS").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data

In [20]:
from autogen import ConversableAgent, register_function

# Create web scrapper agent.
scraper_agent = ConversableAgent(
    "WebScraper",
    llm_config=llm_config_dict,
    system_message="You are a web scrapper and you can scrape any web page using the tools provided. "
    "Returns 'TERMINATE' when the scraping is done.",
)

# Create user proxy agent.
user_proxy_agent = ConversableAgent(
    "UserProxy",
    llm_config=False,  # No LLM for this agent.
    human_input_mode="NEVER",
    code_execution_config=False,  # No code execution for this agent.
    is_termination_msg=lambda x: x.get("content", "") is not None and "terminate" in x["content"].lower(),
    default_auto_reply="Please continue if not finished, otherwise return 'TERMINATE'.",
)

# Register the function with the agents.
register_function(
    scrape_page,
    caller=scraper_agent,
    executor=user_proxy_agent,
    name="scrape_page",
    description="Scrape a web page and return the content.",
)

[autogen.oai.client: 09-20 19:18:59] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 09-20 19:19:00] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [21]:
url = 'https://en.wikipedia.org/wiki/Boise,_Idaho'
chat_result = user_proxy_agent.initiate_chat(
    scraper_agent,
    message=f"Can you scrape {url} for me?",
    summary_method="reflection_with_llm")

UserProxy (to WebScraper):

Can you scrape https://en.wikipedia.org/wiki/Boise,_Idaho for me?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
WebScraper (to UserProxy):

***** Suggested tool call (call_2NOGtisxj5E9ONCx8uYrMU1O): scrape_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Boise,_Idaho"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_page...
UserProxy (to WebScraper):

UserProxy (to WebScraper):

***** Response from calling tool (call_2NOGtisxj5E9ONCx8uYrMU1O) *****
Boise, Idaho - Wikipedia
Boise, Idaho
State capital city
	
Idaho State Capitol
US Bank Building
Eighth & Main
Clearwater Analytics
Jack's Urban Meeting Place
Albertsons Stadium
	
Flag
Seal
Logo
	
Nickname: 
The City of Trees
	
Motto: 
Energy Peril Success
	
Location within Ada County in Idaho


In [22]:
boise_idaho_wiki = chat_result.chat_history[-2]['content']

In [23]:
boise_idaho_wiki

'Boise, Idaho - Wikipedia\nBoise, Idaho\nState capital city\n\t\nIdaho State Capitol\nUS Bank Building\nEighth & Main\nClearwater Analytics\nJack\'s Urban Meeting Place\nAlbertsons Stadium\n\t\nFlag\nSeal\nLogo\n\t\nNickname: \nThe City of Trees\n\t\nMotto: \nEnergy Peril Success\n\t\nLocation within Ada County in Idaho\n\t\nBoise\nLocation within the United States\nShow map of IdahoShow map of the United StatesShow all\n\t\nCoordinates: 43°36′57″N 116°12′6″W / 43.61583°N 116.20167°W\t\nCountryUnited States\t\nStateIdaho\t\nCountyAda\t\nFounded1863\t\nIncorporated1864\t\nGovernment\n• TypeStrong-mayor\t\n• BodyBoise City Council\t\n• MayorLauren McLean (D)\t\n• Council PresidentColin Nash\t\nArea\n[1]\n• State capital city85.00 sq mi (219.45 km2)\t\n• Land84.03 sq mi (216.96 km2)\t\n• Water0.97 sq mi (2.49 km2)\t\nElevation\n[2]\n2,700 ft (823 m)\t\nPopulation\n(2020)[3]\n• State capital city235,684\t\n• RankU.S.: 94th ID: 1st\t\n• Density2,860.54/sq mi (1,055.28/km2)\t\n• Urban433,180

In [24]:
with open("boise_idaho_wiki_content.txt", "w", encoding="utf-8") as file:
    file.write(boise_idaho_wiki)

## Perform RAG on the Scraped Wiki File

In [25]:
# from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

DOC_PATH = ["./boise_idaho_wiki_content.txt"]

In [26]:
llm_config = {'config_list': [{'model': 'gpt-3.5-turbo', 'api_key': os.environ["OPENAI_API_KEY"]}], 'timeout': 60, 'cache_seed': 88, 'temperature': 0}

In [27]:
assistant = AssistantAgent(
    name="assistant",
    human_input_mode="NEVER",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)
        
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": DOC_PATH,
        "chunk_token_size": 1000,
        "model": llm_config_dict["config_list"][0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "boise_idaho",
        "get_or_create": True, # if True, will create/return a collection for the retrieve chat.
    },
    code_execution_config={"use_docker": False}
)

[autogen.oai.client: 09-20 19:19:39] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [28]:
prompt = "What is the population of Boise, Idaho?"
res = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=prompt, n_results=1, silent=True)

2024-09-20 19:19:39,294 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `boise_idaho`.
max_tokens is too small to fit a single line of text. Breaking this line:
		 ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2024-09-20 19:19:39,310 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 7 chunks.


Trying to create collection.
VectorDB returns doc_ids:  [['e495c059']]
Adding content of doc e495c059 to context.


In [29]:
print(f"Question:{prompt}")
print(f"Answer from RAG: {res.chat_history[-1]['content']}")

Question:What is the population of Boise, Idaho?
Answer from RAG: The population of Boise, Idaho is 235,684 as of the 2020 census.


# Group Chat 
Up to this point, we have encountered conversation patterns that include either two agents or a series of two-agent interactions. AutoGen introduces a broader conversation model known as group chat, which includes more than two agents. The fundamental concept of group chat is that all agents participate in a unified conversation thread and share the same context. This approach is beneficial for tasks that necessitate collaboration among several agents.

Add Image Here from Autogen docs :


A group chat is orchestrated by a special agent type GroupChatManager. In the first step of the group chat, the Group Chat Manager selects an agent to speak. Then, the selected agent speaks and the message is sent back to the Group Chat Manager, who broadcasts the message to all other agents in the group. This process repeats until the conversation stops.

The Group Chat Manager can use several strategies to select the next agent. Currently, the following strategies are supported:

round_robin: The Group Chat Manager selects agents in a round-robin fashion based on the order of the agents provided.
random: The Group Chat Manager selects agents randomly.
manual: The Group Chat Manager selects agents by asking for human input.
auto: The default strategy, which selects agents using the Group Chat Manager’s LLM.

TODO : Use Group chat with auto strategy, define multiple agents all having access to web search tool or may be some other tool , each agent has independent persona and contributes to a group chat to create final report for user.


We can create 3 or more agents each having different personas and access to web search tool and a RAG tool? : 
for e.g, Economist persona
Environmentalist persona
Capatalist persona
May be add few more agents with different persona or having access to more tools.

Sample usecases : 
Impacts of asteroid impact on the world. 
Impacts of world war3 on the world. NOTE : We can change the usecase to something else but code structure can stay same.

Create final report for user on above usecase.

In [30]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

from autogen import GroupChat
from autogen import GroupChatManager

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
)
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config_dict,
)

chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

NameError: name 'adder_agent' is not defined

In [ ]:
print(chat_result.summary)

The number 3 was transformed into 13 by adding 4, multiplying by 8, dividing by 1.5, subtracting 2, and then adding 12, dividing by 6.5, subtracting 3, and subtracting 7.


If time and bandwidth permits,we can also do a combination of nested chat with group chat manager. 